# Lab 2.3: Model Architecture Comparison

Objective:
Benchmark VGG16, ResNet50, and MobileNetV2 on:
- Parameter count
- Inference latency
- Estimated FPS

Goal:
Recommend the best model for edge deployment (Raspberry Pi scenario).

In [1]:
import tensorflow as tf
import numpy as np
import time
import pandas as pd

from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2

In [2]:
vgg = VGG16(weights='imagenet')
resnet = ResNet50(weights='imagenet')
mobilenet = MobileNetV2(weights='imagenet')

print("Models loaded successfully.")

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
14536120/14536120 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Models loaded successfully.


In [3]:
models = {
    "VGG16": vgg,
    "ResNet50": resnet,
    "MobileNetV2": mobilenet
}

param_counts = {}

for name, model in models.items():
    params = model.count_params()
    param_counts[name] = params
    print(f"{name}: {params/1e6:.2f} Million parameters")

VGG16: 138.36 Million parameters
ResNet50: 25.64 Million parameters
MobileNetV2: 3.54 Million parameters


In [4]:
dummy_input = np.random.rand(1, 224, 224, 3).astype(np.float32)

In [5]:
def measure_latency(model, input_data, runs=100):
    # Warm-up run
    model.predict(input_data)

    start = time.time()
    for _ in range(runs):
        model.predict(input_data)
    end = time.time()

    avg_time = (end - start) / runs
    return avg_time

In [6]:
latencies = {}

for name, model in models.items():
    avg_time = measure_latency(model, dummy_input, runs=100)
    latencies[name] = avg_time
    print(f"{name}: {avg_time*1000:.2f} ms per inference")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 900ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 559ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 553ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 536ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 952ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 941ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 827ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 557ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 531ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 523ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 551ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 534ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 532ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 550ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 529ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 535ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 574ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [7]:
results = []

for name in models.keys():
    params_m = param_counts[name] / 1e6
    latency_ms = latencies[name] * 1000
    fps = 1 / latencies[name]

    results.append([name, params_m, latency_ms, fps])

df = pd.DataFrame(results, columns=[
    "Model",
    "Parameters (Millions)",
    "Latency (ms)",
    "Estimated FPS"
])

df

,Model,Parameters (Millions),Latency (ms),Estimated FPS
0,VGG16,138.357544,735.154285,1.360259
1,ResNet50,25.636712,326.641855,3.061457
2,MobileNetV2,3.538984,138.983076,7.195121


## Deployment Recommendation

MobileNetV2 is the most suitable architecture for edge deployment. It has significantly fewer parameters and achieves much faster inference speed compared to VGG16 and ResNet50. While VGG16 offers strong accuracy, its large memory footprint and high latency make it unsuitable for Raspberry Pi deployment. MobileNetV2 provides an optimal balance between computational efficiency and classification performance, making it ideal for CPU-based embedded systems.